# Training a general model for a whole dataset

### Here we are going to make a training dataset using the diffraction patterns within all the datasets of the experiment. 

### We are going to select the most information heavy patterns from each dataset and combine them, to form one general training set. We will then train a larger model (a model that has a larger latent space and so is compressing the data less) on this training set, in the hope that the Convolutional layers will learn general trends about these diffraction patterns. We will then transfer the weights of these trained Conv layers to the models for each of our individual datasets and this should speed up the training process

Loading the preamble

In [1]:
#load some packages in
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import random as python_random
from numba import njit
from tensorboard.plugins.hparams import api as hp
from stemutils.io import Path
import hyperspy.api as hs
import concurrent.futures
from skimage.transform import resize
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from functools import lru_cache
from stemseg.processing_funcs import *

#set some variables
print('Using TensorFlow v%s' % tf.__version__)
plt.style.use('default')


#define some functions

###################################################
########### Data Preprocessing ####################
###################################################

def batch_resize(d, bs=512):
    if len(d.shape) == 4:
        flat_d = flatten_nav(d)
    else:
        flat_d = d
    n_batches = int(np.ceil(flat_d.shape[0]//bs))
    batches = [flat_d[i*bs:(i+1)*bs] for i in range(n_batches+1)]
    if len(batches[-1])==0:
        batches.pop(-1)
    print(len(batches[-1]))
    with concurrent.futures.ProcessPoolExecutor(max_workers=4) as exe:
        res = [exe.submit(resize, batch, (batch.shape[0],128,128)) for batch in batches]
    r_batches = [f.result() for f in res]
    return np.concatenate(r_batches, axis = 0).reshape((d.shape[0],128,128))

def data_manip(d, bs = 512):
    if type(d) != np.ndarray:
        print('dask to numpy')
        d = d.compute()
        print('dask to numpy done')
    print('started data manipulations')
    #d = resize(d,(d.shape[0],128,128))
    print('resized')
    d = d.astype('float32')
    for i in range(d.shape[0]):
        d_max = np.max(d[i])
        d[i] = d[i]/d_max
    d = batch_resize(d, bs)
    scaler = np.log(1001)
    return np.log((d*1000)+1)/scaler 


def data_manip_lowq(d, central_box = 128):
    pxc, pyc = d.shape[1]//2, d.shape[2]//2 
    pxl, pxu = pxc - central_box//2, pxc + central_box//2 
    pyl, pyu = pyc - central_box//2, pyc + central_box//2 
    
    d = d[:, pxl:pxu, pyl:pyu]
    if type(d) != np.ndarray:
        print('dask to numpy')
        d = d.compute()
        print('dask to numpy done')
    print('started data manipulations')
    #d = resize(d,(d.shape[0],128,128))
    print('resized')
    d = d.astype('float32')
    for i in range(d.shape[0]):
        d_max = np.max(d[i])
        d[i] = d[i]/d_max
    
    scaler = np.log(1001)
    return np.log((d*1000)+1)/scaler 


###################################################
###################################################
###################################################

def flatten_nav(sig):
    shape = [sig.shape[0]*sig.shape[1]]
    for i in sig.shape[2:]:
        shape.append(i)
    return sig.reshape(shape)


class My_Custom_Generator(keras.utils.Sequence) :
    def __init__(self, image_filenames,  batch_size) :
        self.image_filenames = image_filenames
        self.batch_size = batch_size

    def __len__(self) :
        return (np.ceil(len(self.image_filenames) / float(self.batch_size))).astype(np.int)
    
    
    @lru_cache(None)
    def __getitem__(self, idx) :
        batch_x = self.image_filenames[idx * self.batch_size : (idx+1) * self.batch_size]
        out_img = np.asarray([np.load(file_name)[:,:,None] for file_name in batch_x])
        return out_img, out_img
        #return batch_x, batch_y
        
        
class Array_Generator(keras.utils.Sequence) :
    def __init__(self, images,  batch_size, target = 'same') :
        self.images = images
        self.batch_size = batch_size
        if target == 'same':
            self.target = images
        else:
            self.target = target

    def __len__(self) :
        return (np.ceil(len(self.images) / float(self.batch_size))).astype(np.int)
    
    
    @lru_cache(None)
    def __getitem__(self, idx) :
        out_img = self.images[idx * self.batch_size : (idx+1) * self.batch_size, :,:,None]
        out_targ = self.target[idx * self.batch_size : (idx+1) * self.batch_size, :,:,None]
        return out_img, out_targ
        #return batch_x, batch_y

class Sampling(layers.Layer):
    def call(self, inputs):
        z_mean, z_log_var = inputs
        epsilon = tf.keras.backend.random_normal(shape=tf.shape(z_mean))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon
    
def create_vae_model(hparams):
    
    n_img = 128
    latent_dim = hparams['LAT']
    beta = hparams['B']

    image_input = keras.Input(shape=(n_img, n_img,1), name = 'enc_input')
    x = layers.Conv2D(hparams['KN1'],5, strides = 2, activation='relu',padding='same', input_shape=image_input.shape, name = 'enc_conv1')(image_input)
    x = layers.Conv2D(hparams['KN2'],5, strides = 2, activation='relu',padding='same', name = 'enc_conv2')(x)
    x = layers.Conv2D(hparams['KN3'],5, strides = 2, activation='relu',padding='same', name = 'enc_conv3')(x)
    x = layers.Conv2D(hparams['KN4'],5, strides = 2, activation='relu',padding='same', name = 'enc_conv4')(x)
    x = layers.Conv2D(hparams['KN5'],5, strides = 2, activation='relu',padding='same', name = 'enc_conv5')(x)
    x = layers.Flatten()(x)
    x = layers.Dense(hparams['D1'], activation='relu', name = 'enc_d1')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'enc_d2_t')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'enc_d3_t')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'enc_d4_t')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'enc_d5_t')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'enc_d6_t')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'enc_d7_t')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'enc_d8_t')(x)
    z_mean = layers.Dense(latent_dim, name="z_mean_t")(x)
    z_log_var = layers.Dense(latent_dim, name="z_log_var_t")(x)
    z_output = Sampling()([z_mean, z_log_var])
    encoder_VAE = keras.Model(image_input, [z_mean, z_log_var, z_output])

    z_input = keras.Input(shape=(latent_dim,), name = 'dec_input_t')
    x = layers.Dense(hparams['D2'], activation="relu", name = 'dec_d1_t')(z_input)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'dec_d2')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'dec_d3')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'dec_d4')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'dec_d5')(x)
    x = layers.Dense(hparams['D2'], activation="relu", name = 'dec_d6')(x)
    x = layers.Dense(hparams['D1'], activation="relu", name = 'dec_d7')(x)
    x = layers.Dense(4*4*hparams['KN5'], activation="relu", name = 'dec_d8')(x)
    x = layers.Reshape((4, 4,hparams['KN5']))(x)
    x = layers.Conv2DTranspose(hparams['KN4'],5, strides = 2, activation='relu',padding='same', name = 'dec_conv1')(x)
    x = layers.Conv2DTranspose(hparams['KN3'],5, strides = 2, activation='relu',padding='same', name = 'dec_conv2')(x)
    x = layers.Conv2DTranspose(hparams['KN2'],5, strides = 2, activation='relu',padding='same', name = 'dec_conv3')(x)
    x = layers.Conv2DTranspose(hparams['KN1'],5, strides = 2, activation='relu',padding='same', name = 'dec_conv4')(x)
    image_output = layers.Conv2DTranspose(1,5, strides = 2, activation='sigmoid',padding='same', name = 'dec_conv5')(x)
    #image_output = layers.Conv2DTranspose(16,3, strides = 2, activation='sigmoid',padding='same')
    #image_output = layers.Reshape((n_img, n_img,1))(x)
    decoder_VAE = keras.Model(z_input, image_output)

    # VAE class
    class VAE(keras.Model):
        # constructor
        def __init__(self, encoder, decoder, **kwargs):
            super(VAE, self).__init__(**kwargs)
            self.encoder = encoder
            self.decoder = decoder

        # customise train_step() to implement the loss 
        def train_step(self, x):
            if isinstance(x, tuple):
                x = x[0]
            with tf.GradientTape() as tape:
                # encoding
                z_mean, z_log_var, z = self.encoder(x)
                # decoding
                x_prime = self.decoder(z)
                # reconstruction error by binary crossentropy loss
                reconstruction_loss = tf.reduce_mean(keras.losses.binary_crossentropy(x, x_prime)) * n_img * n_img
                # KL divergence
                kl_loss = -0.5 * tf.reduce_mean(1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
                # loss = reconstruction error + KL divergence
                loss = reconstruction_loss + beta* kl_loss
            # apply gradient
            grads = tape.gradient(loss, self.trainable_weights)
            self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
            # return loss for metrics log
            return {"loss": loss}


        def call(self, x):
            if isinstance(x, tuple):
                x = x[0]
            # encoding
            z_mean, z_log_var, z = self.encoder(x)
            # decoding
            x_prime = self.decoder(z)
            return x_prime
    # build the VAE
    vae_model = VAE(encoder_VAE, decoder_VAE)

    # compile the VAE
    vae_model.compile(optimizer=keras.optimizers.Adam(learning_rate=hparams['LR']),loss=custom_loss)
    vae_model.build((1,128,128,1))
    
    return vae_model



def custom_loss(x,y):
    n_img = 128
    return tf.reduce_mean(keras.losses.binary_crossentropy(x, y)) * n_img * n_img


print(tf.config.list_physical_devices())

2022-08-17 08:29:50.851382: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


Using TensorFlow v2.3.0


/dls_sw/apps/python/anaconda/4.6.14/64/envs/epsic3.7/lib/python3.7/site-packages/pyUSID/viz/__init__.py:18: FutureWarning: Please use sidpy.viz.plot_utils instead of pyUSID.viz.plot_utils. pyUSID.plot_utils will be removed in a future release of pyUSID
  FutureWarning)


Using TensorFlow v2.3.0
Using TensorFlow v2.3.0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU'), PhysicalDevice(name='/physical_device:XLA_GPU:0', device_type='XLA_GPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


2022-08-17 08:30:12.919932: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2022-08-17 08:30:12.921058: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:d8:00.0 name: Tesla V100-PCIE-32GB computeCapability: 7.0
coreClock: 1.38GHz coreCount: 80 deviceMemorySize: 31.75GiB deviceMemoryBandwidth: 836.37GiB/s
2022-08-17 08:30:12.921081: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2022-08-17 08:30:12.921135: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-08-17 08:30:12.921161: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcufft.so.10
2022-08-17 08:30:12.921182: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcurand.so

Getting a list for the paths of the datasets we want to use

In [2]:
fdp = Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure')

dps = [dp for dp in fdp.walk('.hdf5','binned', max_depth =1) if len(fdp.walk('FullModel', max_depth =3)) ==0]

In [4]:
general_training_set = hs.signals.Signal2D(np.zeros((1,128,128)))

In [5]:
gen_train_set_path = Path('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/general_train_set.hdf5')

In [6]:
general_training_set.save(gen_train_set_path, overwrite=True)

Here we are just systematically adding high entropy patterns into one big file, we are doing it sequentially to try and avoid the notebook crashing

In [7]:
for dp in dps:
    print(dp)
    
    gen_train_set = hs.load(gen_train_set_path)

    ### Load in the Data

    sample = ProcessedSample(dp, 'Test')

    ### Create a dictionary to hold some useful information

    info = {}

    nds = flatten_nav(sample.raw_data.data)

    nds.shape

    ds = hs.load(dp)

    sx, sy = ds.data.shape[2:]

    from skimage.measure import shannon_entropy

    entropy = np.zeros((sample.raw_data.data.shape[0:2]))

    for i in range(ds.data.shape[0]):
        for j in range(ds.data.shape[1]):
            entropy[i,j] = shannon_entropy(ds.data[i,j,sx//2-128:sx//2+128, sy//2-128:sy//2+128])

    data_patterns = flatten_nav(ds.data)
    
    ##########

    entr = flatten_nav(entropy**5)
    #This entr array is computing the entropy of each pattern and using that value to define the pdf sl_pdf
    #by changing the power of the entropy you can change how heavily weighted the pdf is for high entropy patterns
    #Higher powers will mean only the most information dense patterns are used, lower powers will allow more low
    #entropy patterns into the training set, 5 should be a fine middle ground
    
    ########

    sl_pdf = entr/ entr.sum()

    ########
    
    num_patts_sampled = 5000
    
    ######
    nldps = []
    sdps = hs.signals.Signal2D(data_patterns[np.random.choice(np.arange(data_patterns.shape[0]), num_patts_sampled, True, sl_pdf)])
    nldps.append(sdps.data)
    nlds = flatten_nav(np.asarray(nldps))




    def data_manip_lowq_resized(d, central_box = 256, bs = 256):
        pxc, pyc = d.shape[1]//2, d.shape[2]//2 
        pxl, pxu = pxc - central_box//2, pxc + central_box//2 
        pyl, pyu = pyc - central_box//2, pyc + central_box//2 

        d = d[:, pxl:pxu, pyl:pyu]
        if type(d) != np.ndarray:
            print('dask to numpy')
            d = d.compute()
            print('dask to numpy done')
        print('started data manipulations')
        #d = resize(d,(d.shape[0],128,128))
        print('resized')
        d = d.astype('float32')
        for i in range(d.shape[0]):
            d_max = np.max(d[i])
            d[i] = d[i]/d_max
        d = batch_resize(d, bs)
        scaler = np.log(1001)
        return np.log((d*1000)+1)/scaler 



    input_data = data_manip_lowq_resized(nlds)
    new_gts = np.concatenate((gen_train_set.data,input_data), axis = 0)
    print(new_gts.shape)
    new_gts_sig = hs.signals.Signal2D(new_gts)
    new_gts_sig.save(gen_train_set_path, overwrite = True) 
    del sample, nds, ds, entropy, data_patterns, entr, sl_pdf, nlds, nldps, input_data, new_gts, new_gts_sig

/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 115625/20210305_115625.hdf5
started data manipulations
resized
136
(5001, 128, 128)
/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 133334/20210305_133334.hdf5
started data manipulations
resized
136
(10001, 128, 128)
/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 135737/20210305_135737.hdf5
started data manipulations
resized
136
(15001, 128, 128)
/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 140302/20210305_140302.hdf5
started data manipulations
resized
136
(20001, 128, 128)
/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/O3_pure/20210305 115108/20210305_115108.hdf5
started data manipulations
resized
136
(25001, 128, 128)
/dls/science/gro

In [15]:
mp = gen_train_set_path.redirect('ModelCheckpoints')

In [16]:
mp.mk()

Now we setup the training dataloader and the model itself

In [17]:
input_data = hs.load(gen_train_set_path).data
shuffle(input_data)
val_data = input_data[::10]

print(input_data.shape)

batch_size = 2048

train_gen = Array_Generator(input_data, batch_size)#, target = input_targets)
valid_gen = Array_Generator(val_data, batch_size)

batch_shape = train_gen[0][0].shape
input_shape = (batch_shape[1],batch_shape[2],batch_shape[3])
out_dims = int(train_gen[0][1].shape[1])

### Check the training data

rand_patt = np.random.randint(0, len(input_data))

### Set the checkpointing

chkpoint_filepath = str(mp)+'/chk-{epoch:02d}-{val_loss:.5e}.hdf5'
chkpoint_model = tf.keras.callbacks.ModelCheckpoint(
    filepath = chkpoint_filepath,
    monitor="val_loss",
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    mode="min",
    save_freq="epoch",
    options=None)

### Redefine the model parameters if you want

rebin_factor = 1

sample_name = 'full'
hparams= {'KN1':32,'KN2':64,'KN3':128, 'KN4':128, 'KN5':256,'D1':128,'D2':512,'LAT':128,'LR':0.0001, 'B':1}

vae_model = create_vae_model(hparams)
info[sample_name] = {'rebin': rebin_factor, 'hparams':hparams}

(85001, 128, 128)


2022-08-17 09:26:42.878673: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-08-17 09:26:42.891429: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 2400000000 Hz
2022-08-17 09:26:42.895405: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x559727d5d020 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-08-17 09:26:42.895436: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-08-17 09:26:42.999114: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55972b6a6f60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-08-17 09:26:42.

In [ ]:
info_path = gen_train_set_path.redirect('general_model_info.json',1)

In [18]:
with open(info_path, 'w') as f:
    json.dump(info,f)

This isn't necessary (and wasn't done for the initial set up of this notebook) but you could load in the weights of a previous initial model to get a better starting point

In [ ]:
vae_model.load_weights('/dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/general_model_weights.hdf5')

Train the model for some number of epochs, the more the better, but the more the longer

In [19]:
### Train the Model

history = vae_model.fit(train_gen, validation_data=valid_gen, epochs=1000, callbacks= [chkpoint_model])

Epoch 1/2500


2022-08-17 09:31:38.840908: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
2022-08-17 09:31:39.111280: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudnn.so.7
2022-08-17 09:31:40.108884: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2022-08-17 09:31:40.215401: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


42/42 [==============================] - ETA: 0s - loss: 8391.7171WARNING:tensorflow:Callbacks method `on_test_batch_end` is slow compared to the batch time (batch time: 0.0359s vs `on_test_batch_end` time: 0.2283s). Check your callbacks.



Epoch 00001: val_loss improved from inf to 3935.88379, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/ModelCheckpoints/chk-01-3.93588e+03.hdf5
42/42 [==============================] - 49s 1s/step - loss: 8293.7048 - val_loss: 3935.8838
Epoch 2/2500
42/42 [==============================] - ETA: 0s - loss: 2777.0357
Epoch 00002: val_loss improved from 3935.88379 to 2353.00610, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/ModelCheckpoints/chk-02-2.35301e+03.hdf5
42/42 [==============================] - 23s 549ms/step - loss: 2771.6490 - val_loss: 2353.0061
Epoch 3/2500
42/42 [==============================] - ETA: 0s - loss: 2231.1217
Epoch 00003: val_loss improved from 2353.00610 to 2146.18481, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModel

Epoch 22/2500
42/42 [==============================] - ETA: 0s - loss: 1958.5836
Epoch 00022: val_loss improved from 1963.05457 to 1962.81323, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/ModelCheckpoints/chk-22-1.96281e+03.hdf5
42/42 [==============================] - 23s 555ms/step - loss: 1959.6300 - val_loss: 1962.8132
Epoch 23/2500
42/42 [==============================] - ETA: 0s - loss: 1959.1425
Epoch 00023: val_loss did not improve from 1962.81323
42/42 [==============================] - 23s 551ms/step - loss: 1955.1432 - val_loss: 1963.7932
Epoch 24/2500
42/42 [==============================] - ETA: 0s - loss: 1957.7998
Epoch 00024: val_loss improved from 1962.81323 to 1959.54333, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/ModelCheckpoints/chk-24-1.95954e+03.hdf5
42/42 [==============================] 

...

Epoch 2281/2500
42/42 [==============================] - ETA: 0s - loss: 1774.1716
Epoch 02281: val_loss improved from 1777.37549 to 1777.15259, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/ModelCheckpoints/chk-2281-1.77715e+03.hdf5
42/42 [==============================] - 23s 554ms/step - loss: 1775.3459 - val_loss: 1777.1526
Epoch 2282/2500
42/42 [==============================] - ETA: 0s - loss: 1773.0543
Epoch 02282: val_loss improved from 1777.15259 to 1776.78955, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/ModelCheckpoints/chk-2282-1.77679e+03.hdf5
42/42 [==============================] - 23s 554ms/step - loss: 1773.9224 - val_loss: 1776.7896
Epoch 2283/2500
42/42 [==============================] - ETA: 0s - loss: 1773.4208
Epoch 02283: val_loss did not improve from 1776.78955
42/42 [======================

Epoch 2346/2500
42/42 [==============================] - ETA: 0s - loss: 1778.6076
Epoch 02346: val_loss did not improve from 1775.57581
42/42 [==============================] - 23s 549ms/step - loss: 1773.7497 - val_loss: 1782.2401
Epoch 2347/2500
42/42 [==============================] - ETA: 0s - loss: 1779.9696
Epoch 02347: val_loss did not improve from 1775.57581
42/42 [==============================] - 23s 551ms/step - loss: 1777.2040 - val_loss: 1781.4878
Epoch 2348/2500
42/42 [==============================] - ETA: 0s - loss: 1778.6324
Epoch 02348: val_loss did not improve from 1775.57581
42/42 [==============================] - 23s 551ms/step - loss: 1785.5211 - val_loss: 1779.0764
Epoch 2349/2500
42/42 [==============================] - ETA: 0s - loss: 1774.9521
Epoch 02349: val_loss did not improve from 1775.57581
42/42 [==============================] - 23s 550ms/step - loss: 1770.6597 - val_loss: 1777.6299
Epoch 2350/2500
42/42 [==============================] - ETA: 0s - l

Epoch 2379/2500
42/42 [==============================] - ETA: 0s - loss: 1776.5062
Epoch 02379: val_loss did not improve from 1774.69189
42/42 [==============================] - 23s 548ms/step - loss: 1779.8988 - val_loss: 1780.7504
Epoch 2380/2500
42/42 [==============================] - ETA: 0s - loss: 1777.2331
Epoch 02380: val_loss did not improve from 1774.69189
42/42 [==============================] - 23s 550ms/step - loss: 1777.4626 - val_loss: 1784.5302
Epoch 2381/2500
42/42 [==============================] - ETA: 0s - loss: 1778.6127
Epoch 02381: val_loss did not improve from 1774.69189
42/42 [==============================] - 23s 552ms/step - loss: 1780.9055 - val_loss: 1777.9464
Epoch 2382/2500
42/42 [==============================] - ETA: 0s - loss: 1775.5825
Epoch 02382: val_loss did not improve from 1774.69189
42/42 [==============================] - 23s 550ms/step - loss: 1771.3458 - val_loss: 1777.3719
Epoch 2383/2500
42/42 [==============================] - ETA: 0s - l

Epoch 2414/2500
42/42 [==============================] - ETA: 0s - loss: 1769.9202
Epoch 02414: val_loss improved from 1773.59253 to 1773.26868, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/ModelCheckpoints/chk-2414-1.77327e+03.hdf5
42/42 [==============================] - 23s 554ms/step - loss: 1776.8528 - val_loss: 1773.2687
Epoch 2415/2500
42/42 [==============================] - ETA: 0s - loss: 1769.8444
Epoch 02415: val_loss improved from 1773.26868 to 1773.15051, saving model to /dls/science/groups/imaging/ePSIC_students/Andy_Bridger/mg28034-1/processing/Merlin/Calibrated/GeneralModels/O3_pure/ModelCheckpoints/chk-2415-1.77315e+03.hdf5
42/42 [==============================] - 23s 553ms/step - loss: 1773.2556 - val_loss: 1773.1505
Epoch 2416/2500
42/42 [==============================] - ETA: 0s - loss: 1770.1393
Epoch 02416: val_loss did not improve from 1773.15051
42/42 [======================

Epoch 2448/2500
42/42 [==============================] - ETA: 0s - loss: 1775.1388
Epoch 02448: val_loss did not improve from 1773.15051
42/42 [==============================] - 23s 550ms/step - loss: 1776.7671 - val_loss: 1776.3983
Epoch 2449/2500
42/42 [==============================] - ETA: 0s - loss: 1773.6388
Epoch 02449: val_loss did not improve from 1773.15051
42/42 [==============================] - 23s 549ms/step - loss: 1772.5068 - val_loss: 1776.0131
Epoch 2450/2500
42/42 [==============================] - ETA: 0s - loss: 1771.1595
Epoch 02450: val_loss did not improve from 1773.15051
42/42 [==============================] - 23s 549ms/step - loss: 1782.9379 - val_loss: 1774.3009
Epoch 2451/2500
42/42 [==============================] - ETA: 0s - loss: 1770.3375
Epoch 02451: val_loss did not improve from 1773.15051
42/42 [==============================] - 23s 549ms/step - loss: 1765.5124 - val_loss: 1773.8730
Epoch 2452/2500
42/42 [==============================] - ETA: 0s - l

These last two cells will delete all the checkpoints other than the lowest val_loss, so run this to clear the directory when done

In [20]:
cps = mp.walk('.hdf5')

In [22]:
cps = mp.walk('.hdf5')
best_model_ind = np.asarray([float(str(i).split('-')[-1].split('.hd')[0]) for i in cps]).argmin()
best_model = cps[best_model_ind]

for x, mod in enumerate(cps):
    if x != best_model_ind:
        mod.unlink()
print('cleared')


cleared
